In [1]:
from IPython.display import Markdown, display

display(Markdown("iotawatt.md"))

### [IoTaWatt](https://iotawatt.com/)

IoTaWatt is an open source electricity hardware monitor that uses current transformers attached to circuits in an electrical box to monitor physical properties of voltage, current and power recording them to a database and viewable thru a local website.  Read more at [IoTaWatt](https://iotawatt.com/).

### Set-up

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(usecwd=True),override=True) # read local .env file

from sqlalchemy import create_engine
from os import environ

username     =  environ.get("POSTGRES_USERNAME", "postgres")
password     =  environ.get("POSTGRES_PASSWORD", "postgres")
ipaddress    =  environ.get("POSTGRES_IPADDRESS", "localhost")
port         =  environ.get("POSTGRES_PORT", "5432")
dbname       =  environ.get("POSTGRES_DBNAME", "ArlingtonMA")

cnx= create_engine(f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')

import pandas as pd
from requests import get

### Extract and Transform

In [ ]:
url = 'http://iotawatt.local/query?'+\
        'select=[time.utc.unix,total_power.wh.d4,Fridge.wh.d4,dryer.wh.d4,oven.wh.d4,Dishwasher.wh.d4,AC.wh.d4]&'+\
        'begin=2023-01-01&end=2023-12-31&group=h&header=yes&limit=None'
raw = get(url).json()
df=pd.DataFrame(raw['data'],columns=raw['labels'])
df.columns = ['timestamp','kwh','fridge','dryer','oven','dishwasher','ac']

df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s').dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)

df = df[~df.timestamp.duplicated()]
df = df[~pd.isnull(df.kwh)]
df = df[df.kwh!=0]

df['pid'] = "YOUR_PID"

for col in ['kwh','fridge','dryer','oven','dishwasher','ac']:
    df[col]=df[col]/1000



### Load

In [ ]:
table_create_query = \
    """
        DROP TABLE IF EXISTS energy.ohms;
        CREATE TABLE energy.ohms (
            pid CHAR(17),
            timestamp  TIMESTAMP,
            kwh REAL,
            fridge REAL,
            dryer REAL,
            oven REAL,
            dishwasher REAL,
            ac REAL,
            PRIMARY KEY(pid,timestamp)
        )
       ;
    """

cnx.execute(table_create_query)

df.to_sql (
        'ohms',
        schema='energy',
        con=cnx,
        if_exists='append',
        index=False
)